# Recommendation 2 Niveaux
---
+ Recommendation de différents genres
+ Recommendation de films du genre choisi

+ **Dataset** : movielens_100k customisé
+ **Algorithme** : Funk SVD / Implicit BPR

### Préparation de 2 datasets
- 1 regroupant la moyenne des notes utiisateurs par genre
- 1 regroupant les notes des films par genre

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix

In [3]:
df_movies = pd.read_csv('../Datasets/movies_clean.csv', index_col=0, engine='python')
df_ratings = pd.read_csv('../Datasets/ml-latest-small/ratings.csv', engine='python').drop('timestamp', axis=1)

In [4]:
df_movies.head()

,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,date
movieId,,,,,,,,,,,,,,,,,,,,,
1,Toy Story,0,1,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1995.0
2,Jumanji,0,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1995.0
3,Grumpier Old Men,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1995.0
4,Waiting to Exhale,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1995.0
5,Father of the Bride Part II,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1995.0


In [5]:
df_ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [6]:
df_genre_ratings = df_ratings.merge(df_movies, on='movieId', how='outer')
df_genre_ratings.head()

,userId,movieId,rating,title,Action,Adventure,Animation,Children,Comedy,Crime,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,date
0,1.0,1,4.0,Toy Story,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1995.0
1,5.0,1,4.0,Toy Story,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1995.0
2,7.0,1,4.5,Toy Story,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1995.0
3,15.0,1,2.5,Toy Story,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1995.0
4,17.0,1,4.5,Toy Story,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1995.0


In [34]:
columns = ['Action', 'Adventure',
       'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

In [30]:
df_genres = df_genre_ratings.groupby('userId' + columns)[['rating']].agg('mean')
df_genres.head()

rating
userId Action Adventure Animation Children Comedy Crime Documentary Drama Fantasy Film-Noir Horror IMAX Musical Mystery Romance Sci-Fi Thriller War Western        
1.0    0      0         0         0        0      0     0           0     0       0         0      0    0       0       0       0      1        0   0           5.0
                                                                                                                1       0       0      1        0   0           5.0
                                                                                            1      0    0       0       0       0      0        0   0           4.0
                                                                                                                                1      0        0   0           4.0
                                                                                                                1       0       0      1        0   0           2.5

In [31]:
df_genres

rating
userId Action Adventure Animation Children Comedy Crime Documentary Drama Fantasy Film-Noir Horror IMAX Musical Mystery Romance Sci-Fi Thriller War Western        
1.0    0      0         0         0        0      0     0           0     0       0         0      0    0       0       0       0      1        0   0          5.00
                                                                                                                1       0       0      1        0   0          5.00
                                                                                            1      0    0       0       0       0      0        0   0          4.00
                                                                                                                                1      0        0   0          4.00
                                                                                                                1       0       0      1        0   0          2.50
...                                                                                                                                                             ...
610.0  1      1         1         1        0      0     0           0     0       0         0      0    0       0       0       1      0        0   0          3.00
                                           1      0     0           0     0       0         0      0    0       0       0       0      0        0   0          4.50
                                                                                                                                                    1          3.50
                                                                                                   1    0       0       0       0      0        0   0          3.50
                                                                          1       0         0      0    0       0       0       0      0        0   0          3.25

[45382 rows x 1 columns]

On a donc les 2 dataframes requis pour réaliser une recommendation à deux niveaux :
+ df_genre_ratings (contient les notes pour chaque films)
+ df_genres (contient la moyenne des notes pour chaque genre)

In [39]:
df_genres.reset_index().query('userId == 2.0')

,userId,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating
121,2.0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,3.000000
122,2.0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,4.500000
123,2.0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,2.500000
124,2.0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,1,0,0,3.500000
125,2.0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,4.500000
126,2.0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,4.000000
127,2.0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,5.000000
128,2.0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,3.000000
129,2.0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,3.500000
130,2.0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.333333


## Recommendation de genres

In [36]:
#Creation de la sparse matrix user/genre
user_genre_matrix = df_genres.reset_index().pivot(index='userId', columns='Action', values='rating')

ValueError: Index contains duplicate entries, cannot reshape